In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

In [2]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

In [3]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

# fume_query(target="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", server="biotech_main", start="2021-12-25T00:00:00.000", end=current_date())

In [4]:
# Arguments: Sash Point, Occ Point, Server Name, Start Time, End Time
# Returns: Total time that hood sash was open when room is unoccupied, aggregated by hour

def total_time_sash_open_unoccupied(sash_point, occ_point, server, start, end):
  sash_master = fume_query(sash_point, server, start, end)
  occ_master = fume_query(occ_point, server, start, end)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  # sash_list.to_csv("test.csv")
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)

  df = pd.concat([sash_list, occ_list], axis=1)
  df.columns = ["sash", "occ"]
  display(df)

  time_interval = df.index[1].minute - df.index[0].minute

  # 1.2 inches is arbitrary
  df["time_open_bool"] = (df["sash"] > 1.2) & (df["occ"] == 0)
  df["time_open_mins"] = np.where(df["time_open_bool"] == False, 0, time_interval)

  df = df.dropna()
  display(df)
  # df.to_csv("test2.csv")

  df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  return df["time_open_mins"]

total_time_sash_open_unoccupied(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

<Response [200]>
<Response [200]>

Sash List:  2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

Sash List new:  2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

OCC List:  2021-11-17 05:00:00    1.0
2021-11-17 05:05:00    1.0
2021-11-17 05:10:00    1.0
2021-11-17 05:15:00    1.0
2021-11-17 05:20:00    1.0
                      ... 
2021-11-18 04:35:00    0.0
2021-11-18 04:40:00    0.0
2021-11-18 04:45:00    0.0
2021-11-18 04:50:00    0.0
2021

,sash,occ
2021-11-17 05:00:00,3.3,1.0
2021-11-17 05:05:00,3.3,1.0
2021-11-17 05:10:00,3.3,1.0
2021-11-17 05:15:00,3.3,1.0
2021-11-17 05:20:00,3.3,1.0
...,...,...
2021-11-18 04:35:00,1.8,0.0
2021-11-18 04:40:00,1.8,0.0
2021-11-18 04:45:00,1.8,0.0
2021-11-18 04:50:00,1.8,0.0


,sash,occ,time_open_bool,time_open_mins
2021-11-17 05:00:00,3.3,1.0,False,0
2021-11-17 05:05:00,3.3,1.0,False,0
2021-11-17 05:10:00,3.3,1.0,False,0
2021-11-17 05:15:00,3.3,1.0,False,0
2021-11-17 05:20:00,3.3,1.0,False,0
...,...,...,...,...
2021-11-18 04:35:00,1.8,0.0,True,5
2021-11-18 04:40:00,1.8,0.0,True,5
2021-11-18 04:45:00,1.8,0.0,True,5
2021-11-18 04:50:00,1.8,0.0,True,5


2021-11-17 06:00:00     0
2021-11-17 07:00:00    10
2021-11-17 08:00:00    50
2021-11-17 09:00:00     0
2021-11-17 10:00:00    30
2021-11-17 11:00:00    40
2021-11-17 12:00:00    15
2021-11-17 13:00:00    40
2021-11-17 14:00:00    10
2021-11-17 15:00:00     0
2021-11-17 16:00:00     0
2021-11-17 17:00:00     0
2021-11-17 18:00:00     0
2021-11-17 19:00:00     0
2021-11-17 20:00:00     0
2021-11-17 21:00:00     0
2021-11-17 22:00:00     0
2021-11-17 23:00:00     0
2021-11-18 00:00:00     0
2021-11-18 01:00:00     0
2021-11-18 02:00:00     0
2021-11-18 03:00:00    55
2021-11-18 04:00:00    25
2021-11-18 05:00:00    30
Freq: 60T, Name: time_open_mins, dtype: int64